## Lecture 12
## Introduction to GPUs (Graphics Processing Units)
### Apr. 26, 2021

Part of this lecture is based on the previous materials, see:

https://nyu-cds.github.io/python-gpu/

https://nyu-cds.github.io/python-numba/05-cuda/

## Architecture
A central processing unit (CPU) is designed to handle complex tasks, emulating virtual machines, control complex flows and branching, security, etc. In contrast, graphical processing units (GPUs) only do one thing well, namely, to handle billions of repetitive low level tasks.

---

Originally designed for the **rendering of triangles** in 3D graphics:
[https://en.wikipedia.org/wiki/Triangle_mesh](https://en.wikipedia.org/wiki/Triangle_mesh)

---


**GPU**s have 1000s of **arithmetic logic units** (ALUs) compared with traditional CPUs that commonly have only 4 or 8. 

---


Many types of scientific algorithms spend most of their time doing just what GPUs are good for: performing billions of repetitive arithmetic operations.


<img src="cpugpuarch.png" alt="Drawing" style="width: 500px;"/>

The following diagram shows how GPU performance has increased compared to traditional CPU architetures along the years.

<img src="01-flops.png" alt="Drawing" style="width: 700px;"/>

#### Difference between a CPU and a GPU
This [video](https://www.youtube.com/watch?v=-P28LKWTzrI) is a funny illustration of the difference, in terms of processing capability, between CPUs and GPUs.

---

When computer scientists first attempted to use GPUs for scientific computing, the scientific codes had to be mapped onto operations designed to render triangles. This was incredibly difficult to do, and took a lot of time and dedication. 

---

Nowadays, there are **high level languages** (such as **CUDA** and **OpenCL**) that target the GPUs directly, so GPU programming is rapidly becoming mainstream in the scientific community.





"OpenCL is an open standard maintained by the non-profit technology consortium Khronos Group. Conformant implementations are available from Altera, AMD, Apple (OpenCL along with OpenGL is deprecated for Apple hardware, in favor of Metal), ARM, Creative, IBM, Imagination, Intel, Nvidia, Qualcomm, Samsung, Vivante, Xilinx, and ZiiLABS."

CUDA is only implemented by Nvidia. 

A **GPU program** comprises two parts: 
1. a *host part* that runs on the CPU: sets up the **parameters** and **data** for the computation
2. one or more *kernels* that run on the GPU: perform the **actual computation**.


## CUDA Programming

The CUDA parallel programming model has **three key abstractions** at its core:
- a hierarchy of thread groups
- shared memories
- barrier synchronization





**Granularity** in parallel programming: amount of computation vs communication.
* **Fine-grained**: individual tasks are relatively small in terms of code size and execution time. The data is transferred among processors frequently in amounts of one or a few memory words.
* **Coarse-grained**: data is communicated infrequently, after larger amounts of computation.

The CUDA abstractions:
* fine-grained data parallelism and thread parallelism (thread blocks)
* coarse-grained data parallelism and task parallelism (grid)

They guide the programmer to partition the problem into coarse sub-problems that can be solved independently in parallel by blocks of threads, and each sub-problem into finer pieces that can be solved cooperatively in parallel by all threads within the block.


- A kernel is executed in parallel by an array of threads:
    - All threads run the same code.
    - Each thread has an ID that it uses to compute memory addresses and make control decisions.

- Threads are arranged as a grid of thread blocks:
    - Different grid/block can have different kernels  
    - Threads from the same block have access to a shared memory and their execution can be synchronized

<img src="threadgrid.png" alt="Drawing" style="width: 300px;"/>



- Thread blocks are required to execute independently: 
    - It must be possible to execute them in any order, in parallel or in series 
    - Threads within a block can cooperate by sharing data through some shared memory and by synchronizing their execution to coordinate memory accesses.
    - The grid of blocks and the thread blocks can be 1, 2, or 3-dimensional.

<img src="threadmapping.png" alt="Drawing" style="width: 300px;"/>

CUDA is designed for a specific GPU architecture, namely NVIDIA’s Streaming Multiprocessors (SM). 
- Each SM has:
    - a set of execution units
    - a set of registers 
    - a chunk of shared memory

<img src="sm.png" alt="Drawing" style="width: 500px;"/>





In an NVIDIA GPU, the basic unit of execution is the __warp__. A warp is a collection of threads, 32 in current implementations, that are executed simultaneously by an SM. Multiple warps can be executed on an SM at once.

When a CUDA program on the host CPU invokes a kernel grid, the blocks of the grid are enumerated and distributed to SMs with available execution capacity. The threads of a thread block execute concurrently on one SM, and multiple thread blocks can execute concurrently on one SM. As thread blocks terminate, new blocks are launched on the vacated SMs.

The mapping between warps and thread blocks can affect the performance of the kernel. It is usually a good idea to keep the size of a thread block a multiple of 32 in order to avoid this as much as possible.





### Thread Identity
The index of a thread and its thread ID relate to each other as follows:
- For a 1-dimensional block, the thread index and thread ID are the same
- For a 2-dimensional block, the thread index (x,y) has thread ID=x+yDx, for block size (Dx,Dy)
- For a 3-dimensional block, the thread index (x,y,x) has thread ID=x+yDx+zDxDy, for block size (Dx,Dy,Dz)

**When a kernel is started, the number of blocks per grid and the number of threads per block are fixed (gridDim and blockDim)**. CUDA makes four pieces of information available to each thread:
- The thread index (threadIdx)
- The block index (blockIdx)
- The size and shape of a block (blockDim)
- The size and shape of a grid (gridDim)

Typically, each thread in a kernel will compute one element of an array. There is a common pattern to do this that most CUDA programs use are shown below.

### CUDA simulator

In [ ]:
# If you do not have a CUDA-enabled GPU on your system, 
# you will receive one of the following errors:

# numba.cuda.cudadrv.error.CudaDriverError: CUDA initialized before forking
# CudaSupportError: Error at driver init: 
# [3] Call to cuInit results in CUDA_ERROR_NOT_INITIALIZED:
# numba.cuda.cudadrv.error.CudaDriverError: Error at driver init:
# CUDA disabled by user:
# If you do have a CUDA-enabled GPU on your system, you should see a message like:

# <Managed Device 0>
# If your machine has multiple GPUs, you might want to select which one to use. 
# By default the CUDA driver selects the fastest GPU as the device 0, 
# which is the default device used by Numba.

# numba.cuda.select_device( device_id )

In [ ]:
# Using the CUDA simulator
# If you don’t have a CUDA-enabled GPU 
# (i.e. you received one of the error messages described previously), 
# then you will need to use the CUDA simulator. 
# The simulator is enabled by setting the environment variable 
# NUMBA_ENABLE_CUDASIM to 1.


# Mac/Linux
# Launch a terminal shell and type the commands:
!export NUMBA_ENABLE_CUDASIM=1

# Windows
# Launch a CMD shell and type the commands:
# SET NUMBA_ENABLE_CUDASIM=1

In [1]:
from numba import cuda
print(cuda.gpus)

cuda.select_device(0)

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [2]:
# %%writefile cuda01.py

from __future__ import division
from numba import cuda
import numpy
import math

# CUDA kernel
@cuda.jit
def my_kernel(io_array):
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    
    index = tx + bx * bw
    io_array[index] = index * 10
    print("i, t, b, w:", index, tx, bx, bw)
        
        
# Host code   
data = numpy.ones(256)
threadsperblock = 16
blockspergrid = math.ceil(data.shape[0] / threadsperblock)

my_kernel[blockspergrid, threadsperblock](data)
print("\ndata:\n", data)

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [ ]:
# ### For a 2-dimensional grid:
# tx = cuda.threadIdx.x
# ty = cuda.threadIdx.y
# bx = cuda.blockIdx.x
# by = cuda.blockIdx.y
# bw = cuda.blockDim.x
# bh = cuda.blockDim.y
# x = tx + bx * bw
# y = ty + by * bh
# array[x, y] = compute(x, y)

### Memory Hierarchy and Data Transfer
The CPU and GPU have separate memory spaces. This means that data that is processed by the GPU must be moved from the CPU to the GPU before the computation starts, and the results of the computation must be moved back to the CPU once processing has completed.

#### Global memory
This memory is accessible to __all threads__ as well as the host (CPU).
- Global memory is allocated and deallocated by the host
- Used to initialize the data that the GPU will work on

#### Shared memory
__Each thread block__ has its own shared memory
- Accessible only by threads within the block
- Much faster than local or global memory
- Requires special handling to get maximum performance
- Only exists for the lifetime of the block

#### Local memory
__Each thread__ has its own private local memory
- Only exists for the lifetime of the thread
- Generally handled automatically by the compiler

#### Constant and texture memory
These are __read-only memory__ spaces accessible by __all threads__.
- Constant memory is used to cache values that are shared by all functional units
- Texture memory is optimized for texturing operations provided by the hardware


## OpenCL and pyOpenCL
__OpenCL__ (Open Computing Language) is an **open standard** for cross-platform, **parallel programming**. It was originally developed by Apple in 2008 and is now maintained by the Khronos Group.

<img src="opencl.png" alt="Drawing" style="width: 600px;"/>
 
While OpenCL supports many different types of processors, as for example GPUs, DSPs, and FPGAs, it is most notably used to access the GPU for general computing tasks.

__pyOpenCL__ is a package (MIT license) that enables developers to easily access the OpenCL API from Python.

A standard and a minimal OpenCL code will have following parts.
1. Identifying a Platform
2. Finding the device ID
3. Creating the context: _to manage objects such as command-queues, memory, program and kernel objects and for executing kernels on one or more devices specified in the context._
4. Creating a command queue in the context
5. Creating a program source and a kernel entry point
6. Creating the buffers for data handling
7. Kernel Program
8. Build and Launch the Kernel
9. Read the Output Buffer and clear it (if needed)

A _pyopencl_ user will have its own device identified by environment variables, simplifying things. Examples can be found [here](https://github.com/inducer/pyopencl/tree/master/examples).

---

**Important:**

See and install pocl to get OpenCL device drivers: https://anaconda.org/conda-forge/pocl

---

In [2]:
# %%writefile info.py

# Find out about your computer's OpenCL situation
import pyopencl as cl  # Import the OpenCL GPU computing API

for platform in cl.get_platforms():  # Print each platform on this computer
    print('=' * 10)
    print('Platform - Name:  ' + platform.name)
    print('Platform - Vendor:  ' + platform.vendor)
    print('Platform - Version:  ' + platform.version)
    print('Platform - Profile:  ' + platform.profile)
    
    for device in platform.get_devices():  # Print each device per-platform
        print('    ' + '-' * 6)
        print('    Device - Name:  ' + device.name)
        print('    Device - Type:  ' + cl.device_type.to_string(device.type))
        print('    Device - Max Clock Speed:  {0} Mhz'.format(device.max_clock_frequency))
        print('    Device - Compute Units:  {0}'.format(device.max_compute_units))
        print('    Device - Local Memory:  {0:.0f} KB'.format(device.local_mem_size/1024))
        print('    Device - Constant Memory:  {0:.0f} KB'.format(device.max_constant_buffer_size/1024))
        print('    Device - Global Memory: {0:.0f} GB'.format(device.global_mem_size/1073741824.0))
print('\n')

Platform - Name:  Apple
Platform - Vendor:  Apple
Platform - Version:  OpenCL 1.2 (Dec 21 2020 17:26:36)
Platform - Profile:  FULL_PROFILE
    ------
    Device - Name:  Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
    Device - Type:  ALL | CPU
    Device - Max Clock Speed:  2300 Mhz
    Device - Compute Units:  8
    Device - Local Memory:  32 KB
    Device - Constant Memory:  64 KB
    Device - Global Memory: 8 GB
    ------
    Device - Name:  Intel(R) Iris(TM) Plus Graphics 655
    Device - Type:  ALL | GPU
    Device - Max Clock Speed:  1050 Mhz
    Device - Compute Units:  48
    Device - Local Memory:  64 KB
    Device - Constant Memory:  64 KB
    Device - Global Memory: 2 GB




In [3]:

# Use OpenCL To Add Two Random Arrays (This Way Hides Details)

import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as pycl_array  # Import PyOpenCL Array 
#(a Numpy array plus an OpenCL buffer object)

import numpy as np  # Import Numpy number tools

# platform = cl.get_platforms()[0]  # Select the first platform [0]
# device = platform.get_devices()[0]  # Select the first device on this platform [0]
# context = cl.Context([device])  # Create a context with your device
context = cl.create_some_context()  # Initialize the Context

print(context)
queue   = cl.CommandQueue(context)  # Instantiate a Queue

# Create two random pyopencl arrays
a = pycl_array.to_device(queue, np.random.rand(50000).astype(np.float32))
b = pycl_array.to_device(queue, np.random.rand(50000).astype(np.float32))  

# Create an empty pyopencl destination array
res_c = pycl_array.empty_like(a)  

program = cl.Program(context, """
__kernel void sum(__global const float *a, __global const float *b, __global float *c)
{
  int i = get_global_id(0);
  c[i] = a[i] + b[i];
}""").build()  # Create the OpenCL program

# Enqueue the program for execution and store the result in c
program.sum(queue, a.shape, None, a.data, b.data, res_c.data)  

print("a: {}".format(a))
print("b: {}".format(b))
print("c: {}".format(res_c))  
# Print all three arrays, to show sum() worked

<pyopencl.Context at 0x7fc6fada75e0 on <pyopencl.Device 'Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz' on 'Apple' at 0xffffffff>>
a: [0.6659326  0.7583772  0.5114547  ... 0.69629675 0.62353325 0.5250863 ]
b: [0.9765464  0.8572221  0.45715094 ... 0.47915205 0.3612234  0.46375078]
c: [1.642479   1.6155993  0.96860564 ... 1.1754488  0.98475665 0.98883706]


---

Example at PyOpenCL's documentation: https://documen.tician.de/pyopencl/


See [https://documen.tician.de/pyopencl/runtime_program.html](https://documen.tician.de/pyopencl/runtime_program.html)

and *associated memory object* `mem_info`: 
[https://documen.tician.de/pyopencl/runtime_const.html#mem_info](https://documen.tician.de/pyopencl/runtime_const.html#mem_info)

---

In [4]:

# the same above algorithm but written in a different way
from time import time
import numpy as np
import pyopencl as cl

n = 5_000_000

a_np = np.random.rand(n).astype(np.float32)
b_np = np.random.rand(n).astype(np.float32)

# ctx = cl.create_some_context()
platform = cl.get_platforms()[0]  # Select the first platform [0]
device = platform.get_devices()[1]  # Select the first device on this platform [0]
ctx = cl.Context([device])  # Create a context with your device
print(ctx)
queue = cl.CommandQueue(ctx)

# Buffer: class pyopencl.Buffer(context, flags, size=0, hostbuf=None)

mf = cl.mem_flags
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)


# get_global_id
# Returns the unique global work-item ID value 
# for dimension identified by dimindx.

prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

ts = time()

res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)

res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

print('Took {}s'.format(time() - ts))

# Check on CPU with Numpy:
print(a_np[0: 10])
print(b_np[0: 10])
print(res_np[0: 10])

print((res_np - (a_np + b_np))[0:10])
print(np.linalg.norm(res_np - (a_np + b_np)))

<pyopencl.Context at 0x7fc6f5254510 on <pyopencl.Device 'Intel(R) Iris(TM) Plus Graphics 655' on 'Apple' at 0x1024500>>
Took 0.052091121673583984s
[0.17456782 0.18152517 0.8936107  0.9087807  0.097303   0.05490942
 0.6503648  0.12005471 0.4902005  0.15932281]
[0.02035814 0.20465158 0.9030432  0.4093773  0.06690929 0.3068401
 0.6603194  0.06385969 0.49010193 0.5361611 ]
[0.19492596 0.38617676 1.796654   1.318158   0.16421229 0.36174953
 1.3106842  0.1839144  0.98030245 0.6954839 ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0


---

pyOpenCL has two goals:
- Make OpenCL seem simple
- Expose OpenCL's complex features

Comparing the two previous codes we see that
```python
context = cl.create_some_context()
```
is simple, but if you have two GPUs in your computer, this function might select the wrong one.  Therefore, you might want to write three lines instead of one:
```python
platform = cl.get_platforms()[0]  # Select the first platform [0]
device = platform.get_devices()[0]  # Select the first device on this platform [0]
context = cl.Context([device])  # Create a context with your device
```
This second way of creating a context is longer, but it allows you to target the exact platform and device you want to use on your machine.